In [1]:
import sys
sys.path.append('../')
from collections import namedtuple
from tqdm import tqdm
import numpy as np
import torch
import torchvision
import src.utils as utils
from src.models import LeNetMNIST
from src.frameworks.online_dvrl import Odvrl

In [ ]:
from collections import namedtuple
Parameters = namedtuple('Parameters', [
    'saving_path',
    'val_batch_size',
    'epochs',
    'device',
    'learning_rate',
    'num_workers',
    'input_dim',
    'hidden_dim',
    'output_dim',
    'layer_number',
])

In [ ]:
parameters = Parameters(
    saving_path='../logs',
    val_batch_size=512,
    epochs=10,
    device='cuda',
    learning_rate=1e-4,
    num_workers=1,
    input_dim=794,
    hidden_dim=1024,
    output_dim=10,
    layer_number=3,
)

In [ ]:
T = 10
noise_level = 0.2
seed = 6
num_weak = 30
pred_model = LeNetMNIST()
val_model = LeNetMNIST()

In [ ]:
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
# run 'pretrain_mnist.ipynb' first
# then load this pretrained model
# state_dict = torch.load("../data/pretrained_models/pretrained_lenetmnist.pt")
# val_model.load_state_dict(state_dict)

In [ ]:
x_train, y_train, x_test, y_test, noisy_idxs = utils.create_noisy_mnist(method='uniform', noise_level=noise_level)

In [ ]:
x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)
# test_data = torch.utils.data.TensorDataset(torch.tensor(x_test), torch.tensor(y_test))
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,))
])
test_data = torchvision.datasets.MNIST('../data', train=False, download=True, transform=transform)

In [ ]:
num_data = len(y_train)

In [ ]:
engine = Odvrl(num_weak=num_weak, pred_model=pred_model, val_model=val_model, parameters=parameters)

In [ ]:
subset_len = num_data // T
for t in range(T):
    start_id = t * subset_len
    end_id = min((t + 1) * subset_len, num_data)
    engine.one_step(
        t, 
        X=x_train[start_id:end_id], 
        y=y_train[start_id:end_id], 
        val_dataset=test_data, 
    )
    current_noisy_idxs = np.extract((noisy_idxs >= start_id) & (noisy_idxs < end_id), noisy_idxs)
    current_corrupted_num = len(current_noisy_idxs)
    if current_corrupted_num > 0:
        guess_idxs = np.argsort(engine.evaluate(x_train, y_train))
        discover_rate = len(np.in1d(guess_idxs, current_noisy_idxs)) / current_corrupted_num
        print('discover rate: {}'.format(discover_rate))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [20, 1, 5, 5], but got 3-dimensional input of size [60000, 28, 28] instead